In [79]:
class Neuron(object):
    
    def __init__(self, set_of_input_weights, set_of_inputs, threshold):
        
        self.threshold = threshold
               
        self.set_of_input_weights = set_of_input_weights
        self.set_of_inputs = set_of_inputs
        self.num_of_inputs = len(self.set_of_inputs)   
        
    def set_threshold(self, value):
        self.threshold = value
    
    def output(self):
        length = self.num_of_inputs
        activation = 0
        output = 0
        for i in range(length):
            activation += self.set_of_input_weights[i] * self.set_of_inputs[i]
#             print "Activation: ", activation
        if activation > self.threshold:
            output = 1
        return output



In [80]:
weights = [0.5, 0.1, 0.4]
inputs = [1, 0, 1]
threshold = 0.5
n = Neuron(weights, inputs, 0.5)
# print n.output()


In [81]:
weights = {
    0: 0.01,
    1: 0.02,
    2: 0.1,
    3: 0.5,
    4: -0.6,
    5: 1,
    6: 1.5,
    7: 0.8,
    8: 0.05,
    9: -0.01,
    10: 0.005,
    11: 0.1,
    12: 0.5,
}
thresholds = {
    0: 0.05,
    1: 0.1,
    2: 0,
    3: 0.5,
    4: 1,
    5: 0.5,
}
inputs = [1, 0]
n0 = Neuron([weights[0], weights[3]], inputs, thresholds[0])
n1 = Neuron([weights[1], weights[2]], inputs, thresholds[1])
n2 = Neuron([weights[4], weights[7]], [n0.output(), n1.output()], thresholds[2])
n3 = Neuron([weights[5], weights[8]], [n0.output(), n1.output()], thresholds[3])
n4 = Neuron([weights[6], weights[9]], [n0.output(), n1.output()], thresholds[4])
n5 = Neuron([weights[10], weights[11], weights[12]], [n2.output(), n3.output(), n4.output()], thresholds[5])
# print n5.output()

In [82]:
import random as r


def init_weight_zero_population(size):
    """
    in: desired size of population
    out: first random population
    """
    population = []
    for i in range(size):
        # from 0 to 40 1's
        population.append(r.randrange(0, 1099511627775))
    return population

def init_weight_one_population(size):
    """
    in: desired size of population
    out: first random population
    """
    population = []
    for i in range(size):
        # from 0 to 40 1's
        population.append(r.randrange(0, 1099511627775))
    return population

def init_weight_two_population(size):
    """
    in: desired size of population
    out: first random population
    """
    population = []
    for i in range(size):
        # from 0 to 24 1's
        population.append(r.randrange(0, 16777215))
    return population

def init_theshold_population(size):
    """
    in: desired size of population
    out: first random population
    """
    population = []
    for i in range(size):
        # from 0 to 48 1's
        population.append(r.randrange(0, 281474976710655))
    return population


In [83]:

def decode_weight_zero(chromosome):
    """
    decodes the weight zero population.
    1 of 3 weight populations.
    """
    w0 = (chromosome & 255)
    w1 = ((chromosome & 65280) >> 8)
    w2 = ((chromosome & 16711680) >> 16)
    w3 = ((chromosome & 4278190080) >> 24)
    w4 = ((chromosome & 1095216660480) >> 32)
    weights = [w0, w1, w2, w3, w4]
    return weights

def decode_weight_one(chromosome):
    """
    decodes the weight one population.
    2 of 3 weight populations.
    """
    w5 = (chromosome & 255)
    w6 = ((chromosome & 65280) >> 8)
    w7 = ((chromosome & 16711680) >> 16)
    w8 = ((chromosome & 4278190080) >> 24)
    w9 = ((chromosome & 1095216660480) >> 32)
    weights = [w5, w6, w7, w8, w9]
    return weights
    
def decode_weight_two(chromosome):
    """
    decodes the weight two population.
    3 of 3 weight populations.
    """
    w10 = (chromosome & 255)
    w11 = ((chromosome & 65280) >> 8)
    w12 = ((chromosome & 4278190080) >> 24)
    weights = [w10, w11, w12]
    return weights
    
def decode_thresholds(chromosome):
    """
    decodes the population that represent the thresholds.
    returns: 6 threshold values (-255 - 255)
    """
    t0 = (chromosome & 255)
    t1 = ((chromosome & 65280) >> 8)
    t2 = ((chromosome & 16711680) >> 16)
    t3 = ((chromosome & 4278190080) >> 24)
    t4 = ((chromosome & 1095216660480) >> 32)
    t5 = ((chromosome & 280375465082880) >> 40)
    thresholds = [t0, t1, t2, t3, t4, t5]
    return thresholds


In [84]:
def fitness(ann_data):
    weights = ann_data["w"]
    thresholds = ann_data["thresholds"]
    inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
    results = [0, 1, 1, 0]
    value = 0
    for i in range(4): # four posssible right answers
        if run_network(weights, thresholds, inputs[i]) == results[i]:
            value += 1
    return (10 - abs(value - 4))
    
def run_network(weights, thresholds, inputs):
    n0 = Neuron([weights[0], weights[3]], inputs, thresholds[0])
    n1 = Neuron([weights[1], weights[2]], inputs, thresholds[1])
    n2 = Neuron([weights[4], weights[7]], [n0.output(), n1.output()], thresholds[2])
    n3 = Neuron([weights[5], weights[8]], [n0.output(), n1.output()], thresholds[3])
    n4 = Neuron([weights[6], weights[9]], [n0.output(), n1.output()], thresholds[4])
    n5 = Neuron([weights[10], weights[11], weights[12]], [n2.output(), n3.output(), n4.output()], thresholds[5])
    return n5.output()

global container
def best_yet(ann_data, result):    
    if container.get(result, 0) == 0:
        container[ann_data] = result
        
        
        
        
def is_solution(ann_data):
    if fitness(ann_data) == 10:
        return True
    return False

In [85]:
def generations(gen_size):
    """
    """
    
    w0_population = init_weight_zero_population(5)
    w1_population = init_weight_one_population(5)
    w2_population = init_weight_two_population(5)
    t_population = init_theshold_population(5)
    gen = 0
    population_size = 1
    for i in range(gen_size): # generations
        temp_w0_population = []
        temp_w1_population = []
        temp_w2_population = []
        temp_t_population = []
        for index in range(population_size):
            w0chromosome = w0_population[index]
            w1chromosome = w1_population[index]
            w2chromosome = w2_population[index]
            thresholdchromosome = t_population[index]
            w0 = decode_weight_zero(w0chromosome)
            w1 = decode_weight_one(w1chromosome)
            w2 = decode_weight_two(w2chromosome)
            tresholds = decode_thresholds(thresholdchromosome)
            neural_net_weights_and_thresholds = {
                "w": w0 + w1 + w2, "thresholds": tresholds}
            print neural_net_weights_and_thresholds
            if is_solution(neural_net_weights_and_thresholds):
                print "SOLUTION FOUND"
                return
            def create_new_child(chromosome, population):
                parents = [chromosome, select_partner(population)]
                crossed_child = crossover(parents)
                mutated_child = mutate(crossed_child)
                return mutated_child
            temp_w0_population.append(create_new_child(w0, w0_population))
            temp_w1_population.append(create_new_child(w1, w1_population))
            temp_w2_population.append(create_new_child(w2, w2_population))
            temp_t_population.append(create_new_child(t, t_population))
        gen += 1
        print "Generation: ", gen
        print "Maximum Fitness Score: ", max_fitness(population)
        print "Average Fitness Score: ", average_fitness(population)
        w0_population = temp_w0_population
        w1_population = temp_w1_population
        w2_population = temp_w2_population
        t_population = temp_t_population

generations(1)



{'thresholds': [19, 95, 38, 238, 41, 136], 'w': [243, 13, 206, 17, 6, 102, 191, 74, 3, 148, 49, 252, 0]}


TypeError: 'int' object has no attribute '__getitem__'

In [ ]:
def select_partner(population):
    """
    in: population
    out: index of population selected for breeding
    method: roulette
    """
    fits = [fitness(x) for x in population]
    fit_sum = sum(fits)
    rnd = r.randrange(0, fit_sum)
    big_list = []
    for p in population:
        for f in xrange(fitness(p)):
            big_list.append(p)
    # print "big_list: ", big_list
    return big_list[rnd]


def crossover(parents):
    """
    in: parents [2]
    out: children [1]
    rate: 0.5
    """
    def get_first_half(c):
        return (c & 75557863725914323419135)
    def get_second_half(c):
        return (c & 5708990770823839524233068319934254631207567360)
    # crossover_decision = r.randrange(0, 1)
    crossover_decision = 1
    if crossover_decision:
        male_first_half = get_first_half(parents[0])
        female_second_half = get_second_half(parents[1])
        child = (male_first_half | female_second_half)
        return child
    else: return parents

def mutate(chromosome):
    """
    in: chromosome
    out: mutated chromosome
    rate: 1
    *
      note if rate is 0.5 takes significantly
      longer to find solution.
    *
    """
    bit_to_flip = r.randrange(0, 152)
    mutated_chrome = chromosome ^ (1 << bit_to_flip)
    return mutated_chrome

def average_fitness(population):
    """
    in: population
    out: sum of all fitness scores / length of population
    """
    fits = [fitness(x) for x in population]
    return (sum(fits)/len(fits))

def max_fitness(population):
    fits = [fitness(x) for x in population]
    return max(fits)

In [91]:
x = {}
x[4] = [123, 123, 123, 123, 123]
print x

if 4 in x:
    print "yes"

{4: [123, 123, 123, 123, 123]}
yes
